In [29]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt

In [23]:
def colorDectect(img_path, option = 0):
    img = cv2.imread(img_path)
    img1 = img2 = img.copy()
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)  ### 把RGB颜色转化为HSV颜色
    img = cv2.GaussianBlur(img, (5,5), 0)    ### 高斯滤波图片 （5，5）卷积核
    
    cv2.imshow('GaussianBlur', img)
    
    if option == 0:  ### 选取蓝色
        lower = np.array([100, 50, 50]) ## 蓝色低阀值
        upper = np.array([124, 255, 255])  ## 蓝色高阀值
    if option == 1: ### 选取黄色
        lower = np.array([15, 50, 50])
        upper = np.array([40, 255, 255])
    
    mask = cv2.inRange(hsv, lower, upper)  ##构建阀值掩模
    res = cv2.bitwise_and(img, img, mask = mask) ##根据阀值掩模进行与运算
    cv2.imshow('bitwise_and', res)
    gray = cv2.cvtColor(res, cv2.COLOR_RGB2GRAY) ## 转为灰色
    ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)  ## 二值化
    cv2.imshow('threshold', thresh)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (17, 3))
    closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)  ## 闭操作
    (cnts, _) = cv2.findContours(closed.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    imgRs = []
    i = 0
    for cnt in cnts:
        rect = cv2.minAreaRect(cnt)
        x,y,w,h = cv2.boundingRect(cnt)
        if (w<50 or h < 15 or w>h < 1.0):
            continue
        cv2.rectangle(img1,(x,y),(x+w,y+h),(0,255,0),1)
        #imgCrop = _colorImage[y:y+h,x:x+w]
        imgRs.append((x,y,w,h,rect[2]))
        rs = img[y:y+h,x:x+w]
        #cv2.imshow("============="+str(name),rs)

    #cv2.drawContours(_colorImage, [_box], -1, (0,0,255), 1)
    #cv2.imshow("_colorImage",_colorImage)

    return imgRs, img1 

In [24]:
rects, img = colorDectect('F:/opencv/carnumber/luHH9999.jpg', option= 1)

cv2.imshow('test', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [36]:
def preprocess(gray):
    guassian = cv2.GaussianBlur(gray, (3, 3), 0, 0, cv2.BORDER_DEFAULT)  ## 高斯平滑
    median = cv2.medianBlur(guassian, 5)           ## 中值过滤
    sobel = cv2.Sobel(median, cv2.CV_8U, 1, 0, ksize = 3)  ## sobel算子 x方向梯度
    ret, binary = cv2.threshold(sobel, 170, 255, cv2.THRESH_BINARY)
    element1 = cv2.getStructuringElement(cv2.MORPH_RECT, (9, 1))
    element2 = cv2.getStructuringElement(cv2.MORPH_RECT, (9, 7))
    dilation = cv2.dilate(binary, element2, iterations = 1)  ## 膨胀
    erosion = cv2.erode(dilation, element1, iterations = 1)  ## 腐蚀
    dilation1 = cv2.dilate(erosion, element2, iterations = 3)  ## 再次膨胀
    
    cv2.imshow('dliation', dilation)
    cv2.imshow('erosion', erosion)
    cv2.imshow('dilation1', dilation1)
    
    return dilation1
    

In [26]:
def findPlateRegion(img):
    region = []
    contours, hierarchy = cv2.findContours(img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    ### 筛选面积
    for cnt in contours:
        area = cv2.contourArea(cnt)
        ## 面积小于2000过滤掉
        if (area < 2000):
            continue
        
        epsilon = 0.001 * cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, epsilon, True)  ## 轮廓近似值
        
        rect = cv2.minAreaRect(cnt)
        print('rect is: ', rect)
        
        boxer = cv2.boxPoints(rect)
        boxer = np.int0(boxer)
        # 计算高和宽
        height = abs(boxer[0][1] - boxer[2][1])
        width = abs(boxer[0][0] - boxer[2][0])
        # 车牌正常情况下长高比在2.7-5之间
        ratio =float(width) / float(height)
        print('ration: ',ratio)
        if (ratio > 5 or ratio < 2):
            continue
        region.append(boxer)
    
    return region
       
            

In [27]:
def detect(img):
    ## 灰度化
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ## 形态学变换
    dilation = preprocess(gray)
    ## 查找车牌区域
    region = findPlateRegion(dilation)
    
    for boxer in region:
        cv2.drawContours(img, [boxer], 0, (0, 255, 0), 2)
    
    ys = [boxer[0, 1], boxer[1, 1], boxer[2, 1], boxer[3, 1]]
    xs = [boxer[0, 0], boxer[1, 0], boxer[2, 0], boxer[3, 0]]
    ys_sort_idx = np.argsort(ys)
    xs_sort_idx = np.argsort(xs)
    
    x1 = boxer[xs_sort_idx[0], 0]
    x2 = boxer[xs_sort_idx[3], 0]
    
    y1 = boxer[ys_sort_idx[0], 1]
    y2 = boxer[ys_sort_idx[3], 1]
    
    img_org = img.copy()
    img_plate = img_org[y1:y2, x1:x2]
    cv2.imshow('number plate', img_plate)
    cv2.imwrite('F:/opencv/carnumber/number_plate.jpg', img_plate)
    
    cv2.namedWindow('img', cv2.WINDOW_NORMAL)
    cv2.imshow('img', img)
    cv2.imwrite('F:/opencv/carnumber/contours.png', img)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [37]:
img = cv2.imread('F:/opencv/carnumber/luHH9999.jpg')
detect(img)

rect is:  ((452.5, 306.0), (48.0, 93.0), -90.0)
ration:  1.9375
rect is:  ((277.57769775390625, 258.57208251953125), (219.65811157226562, 72.7613525390625), -2.147585391998291)
ration:  2.6666666666666665
rect is:  ((279.0, 73.5), (51.0, 58.0), -90.0)
ration:  1.1372549019607843
rect is:  ((27.365718841552734, 45.41130065917969), (98.78160858154297, 39.3486328125), -69.94390869140625)
ration:  0.02857142857142857
